In [1]:
computer_issu_or_question_list = ''' \
a computer software, \
Windows, \
Printer, \
Mouse, \
Micrphone, \
Heaset, \
Sound, \
Battery issues, \
PC heating up, \
Game not working, \
Game makes computer slow, \
Display and Appearance issues, \
Sound and Audio issues, \
Network and Internet issues, \
Power and Battery, \
Privacy and Security, \
Accessibility, \
Keyboard and Mouse issues, \
System Updates and Maintenance issues, \
File and Folder, \
Applications and Software issues, \
Slow performance ,\
Computer crashes or freezes ,\
'''

In [ ]:
prompt_template = "A model that takes in a description of a computer related issue and how to use {computer_issue_list}, and other general computer issues doesn't show in here \
or feature in English, and responds with the troubleshooting guide and guide or tutorial in English."

prompt = prompt_template.format(computer_issue_list=computer_issu_or_question_list)

print(prompt)

In [ ]:
temperature = .4
# temperature = .2

number_of_examples = 50
# number_of_examples = 10

import os
import openai
import random

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [ ]:
def generate_example(prompt, prev_examples, temperature=.5):
    messages=[
        {
            "role": "system",
            "content": f"You are generating data which will be used to train \
a machine learning model.\n\nYou will be given a high-level description of the model \
we want to train, and from that, you will generate data samples, each with a prompt/response pair.\
\n\nYou will do so in this format:\n```\nprompt\n-----------\n$prompt_goes_here\n-----------\
\n\nresponse\n-----------\n$response_goes_here\n-----------\n```\n\nOnly one prompt/response pair should be generated per turn.\
\n\nFor each turn, make the example slightly more complex than the last, while ensuring diversity.\
\n\nMake sure your samples are unique and diverse, yet high-quality and complex enough to train a well-performing model.\
\n\nHere is the type of model we want to train:\n`{prompt}`"
        }
    ]

    if len(prev_examples) > 0:
        if len(prev_examples) > 10:
            prev_examples = random.sample(prev_examples, 10)
        for example in prev_examples:
            messages.append({
                "role": "assistant",
                "content": example
            })

    '''
    ** gpt-3.5-turbo-16k-0613
    Snapshot of gpt-3.5-turbo-16k from June 13th 2023. Unlike gpt-3.5-turbo-16k, 
    this model will not receive updates, and will be deprecated 3 months after a new version is released.
    max token: 16,385 tokens
    '''
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k-0613", # gpt-4
        messages=messages,
        temperature=temperature,
        max_tokens=4096, # 1354 , 500, max : 4096
    )

    return response.choices[0].message['content']

# Generate examples
prev_examples = []
for i in range(number_of_examples):
    print(f'Generating example {i}')
    example = generate_example(prompt, prev_examples, temperature)
    prev_examples.append(example)
    # print index of example and example
    print(f'Example {i} generated: {example}')

print(prev_examples)

In [ ]:
def generate_system_message(prompt):
    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k-0613", # gpt-4
        messages=[
          {
            "role": "system",
            "content": "You will be given a high-level description of the model we are training, \
and from that, you will generate a simple system prompt for that model to use. \
Remember, you are not generating the system message for data generation -- \
you are generating the system message to use for inference. \
A good format to follow is `Given $INPUT_DATA, you will $WHAT_THE_MODEL_SHOULD_DO.`.\
\n\nMake it as concise as possible. Include nothing but the system prompt in your response.\
\n\nFor example, never write: `\"$SYSTEM_PROMPT_HERE\"`.\n\nIt should be like: `$SYSTEM_PROMPT_HERE`."
          },
          {
              "role": "user",
              "content": prompt.strip(),
          }
        ],
        temperature=temperature,
        max_tokens=500,
    )

    return response.choices[0].message['content']

system_message = generate_system_message(prompt)

print(f'The system message is: `{system_message}`. \nFeel free to re-run this cell if you want a better result.')

In [ ]:
import pandas as pd

# Initialize lists to store prompts and responses
prompts = []
responses = []

# Parse out prompts and responses from examples
for example in prev_examples:
  try:
    split_example = example.split('-----------')
    prompts.append(split_example[1].strip())
    responses.append(split_example[3].strip())
  except:
    pass

# Create a DataFrame
df = pd.DataFrame({
    'prompt': prompts,
    'response': responses
})

# Remove duplicates
df = df.drop_duplicates()

print('There are ' + str(len(df)) + ' successfully-generated examples. Here are the first few:')

df.head()

In [ ]:
# Rename the columns
df = df.rename(columns={
    'prompt': 'Question',
    'response': 'Solution'
})

# Save DataFrame to CSV without headers first
file_path = 'data/gpt-gen-data-llm.csv'
df.to_csv(file_path, header=False, index=False)

df = df.rename(columns={
    'prompt': 'Question',
    'response': 'Solution'
})

# save df to csv with headers. need to add First column is 'prompt', second column is 'response'
df.to_csv('data/gpt-gen-data-llm.csv', index=False, header=False)

In [ ]:
# Split the data into train and test sets, with 90% in the train set
train_df = df.sample(frac=0.9, random_state=42)
test_df = df.drop(train_df.index)

# Save the dataframes to .jsonl files
train_df.to_json('data/train.jsonl', orient='records', lines=True)
test_df.to_json('data/test.jsonl', orient='records', lines=True)